In [11]:

from os import listdir
from os.path import isfile, join
import os
import random
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import torch
from torchvision.utils import save_image
import secrets
import shutil
import re

In [2]:
def split_images_randomply(path):
    print("Splitting")
    files = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    random.shuffle(files)
    train, validate, test = np.split(files, [int(len(files)*0.8), int(len(files)*0.9)])
    return train, validate, test


def augment_data_and_save(dataset_files, path):
    print("Transforming")
    transformer = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(640),
        transforms.CenterCrop(640),
        transforms.ToTensor()
    ])

    random.shuffle(dataset_files)

    for image_path in dataset_files:
        target_img = torchvision.io.read_image(image_path)

        save_image(transformer(target_img), path + os.path.basename(image_path))


In [3]:
fs_train, fs_validate, fs_test = split_images_randomply(r"./raw_dataset/images/")
augment_data_and_save(fs_train, r'./images/train/')
augment_data_and_save(fs_validate, r'./images/vals/')
augment_data_and_save(fs_test, r'./images/test/')



Splitting
Transforming
Transforming
Transforming


In [12]:
def modify_files_in_directory(directory, new_value):
    # Loop through each file in the directory
    for filename in os.listdir(directory):
        # Skip any files that do not have the expected format
        if not filename.endswith(".txt"):
            continue

        # Read the contents of the file
        with open(os.path.join(directory, filename), "r") as f:
            lines = f.readlines()

        # Modify each line to replace the first column with the new value and remove numeric values from the last column
        modified_lines = []
        for line in lines:
            line_parts = line.split(" ")
            line_parts[0] = new_value
            last_part = line_parts[-1].strip()
            if re.match(r"^\d+(\.\d+)?$", last_part):
                line_parts[-1] = "\n"
            modified_line = " ".join(line_parts)
            modified_lines.append(modified_line)

        # Write the modified contents back to the file
        with open(os.path.join(directory, filename), "w") as f:
            f.writelines(modified_lines)

In [13]:
modify_files_in_directory("./labels/train/", "0")
modify_files_in_directory("./labels/test/", "0")
modify_files_in_directory("./labels/vals/", "0")